# Re-center Particles

How to update particle location centers from alignments in 2D Classification data. 

Before proceeding, please [ensure the `cryosparc-tools` module is installed]('../intro') in your Python environment. Also ensure cryoSPARC base ports +2 and +3 (`39002` and `39003` in this example) are available on this machine.

First initialize the `CryoSPARC` client:

In [ ]:
from cryosparc.tools import CryoSPARC

cs = CryoSPARC(port=39000)
assert cs.test_connection()

This instance has a 2D Classification and Select 2D Classes job at `P70-J4` and `P70-J5`, respectively. We want to re-extract those selected particles with updated centers computed by the 2D Classification job.

Retrieve the job handle and its particles output with the `load_output` method.

In [ ]:
project = cs.find_project("P2")
job = cs.find_job("P2", "J1077")
particles = job.load_output('particles_selected')

Get a subset of relevant columns and display the first 10 rows as a `pandas` dataframe:

In [ ]:
import pandas as pd

first_10 = particles.slice(0, 10)
first_10 = first_10.filter_prefixes(['alignments2D', 'location'])
pd.DataFrame(first_10.rows())

Each particle has [`location/center_x_frac`, `location/center_y_frac`] fields which contain the fractional distance of each particle from the top-left corner of the micrograph. By convention, the top-left corner of the micrograph is `[0, 0]` and the bottom right corner is `[1, 1]`.

Use the shifts calculated by the 2D Classification job to modify the particle location. These are located at `alignments2D/shift` for each particle. These units are in pixels. Convert the original locations to pixels before doing the shift operation, and then back to fractions for increased floating point precision.

Be sure to update the `mic_psize` to the pixel size of the imported micrographs.

In [ ]:
mic_psize = 0.6575  # import size of micrographs (may differ from particles if binned)
shift_x = particles['alignments2D/psize_A'] * particles['alignments2D/shift'][:, 0] / mic_psize
shift_y = particles['alignments2D/psize_A'] * particles['alignments2D/shift'][:, 1] / mic_psize

Let's plot these particles onto our of micrographs. First download the micrograph of one of the particles.

In [ ]:
mic_path = particles["location/micrograph_path"][0]
header, mic = project.download_mrc(mic_path)

Downsample and add a low-pass filter with cryoSPARC's included utilities.

Use `matplotlib` overlay the previous and new particle locations. These are shown in yellow and blue below, respectively.

In [ ]:
from cryosparc.util import downsample, lowpass
import matplotlib.pyplot as plt
import numpy as np

binned = downsample(mic, factor=4)
lowpassed = lowpass(binned, psize=mic_psize, amount=20, order=0.7)

vmin = np.percentile(lowpassed, 1)
vmax = np.percentile(lowpassed, 99)
fig, ax = plt.subplots(dpi=300)
ax.axis("off")
ax.imshow(lowpassed, cmap="gray", vmin=vmin, vmax=vmin)

fig.tight_layout()
fig.show()

In [ ]:
# EXAMPLE CUSTOM JOB/RESULTS
cs.upload_results_dataset( 'particles', particles, schema={
    'parent': 'P2-J42', 
    'type': 'particles', 
    'parent': 'J42', 
    'fields': ['alignments2D']})

job = CustomJob('import_result_group')
job.add_output('particles', fields=['location', ('map_sharp', 'volume')])

with job.run():
    job.set_output('particles', particles)


